# ADAU1401.Control Functional Test

https://www.analog.com/media/en/technical-documentation/data-sheets/adau1401.pdf

In [1]:
from pathfinder import Pathfinder

# abs_paths = ['']
# relative_paths = Pathfinder.relative_paths_from_abs(abs_paths)

relative_paths = [['..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Signal Generators', 'bitbucket', 'github', 'codes'],
                  ['..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
                  ['..', '..', 'codes']]

Pathfinder.append_relative_paths(relative_paths)

In [2]:
import pandas as pd

#https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

In [3]:
from bridges.ftdi.adapters.micropython import machine
from bridges.ftdi.controllers.i2c import I2cController
from sigma.sigma_dsp.adau import ADAU1401
from utilities.adapters import peripherals

## Debug mode?

In [4]:
cls = ADAU1401

cls.DEBUG_MODE_SHOW_BUS_DATA = False         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## DSP processor

In [5]:
with_hardware_device = False  # set True if hardware is connected.

if with_hardware_device:
    
    ctrl = I2cController()
    _machine = ctrl.get_gpio()

    _i2c = ctrl.I2C()

    _pin_reset = _machine.Pin('ADBUS4', mode = machine.Pin.OUT)
    _pin_reset.high()

else:
    _i2c = _pin_reset = None  # using None for testing without actual hardware device.

bus = peripherals.I2C(_i2c)


****** Virtual device. Data may not be real ! ******



In [6]:
# bus = fx2lp.I2C(as_400KHz = False)

In [7]:
dsp = ADAU1401(bus, pin_reset = _pin_reset)

## Methods Test

### [Message](https://github.com/Wei1234c/SigmaDSP/blob/master/codes/sigma/sigma_dsp/messages.py) related methods

In [8]:
# load messages bytes from binary file

with open('E2Prom.bin', 'br') as f:
    messages_bytes = f.read()

In [9]:
# parse data bytes in to a collections of "Message" object.

messages = dsp.control.messages_from_bytes(messages_bytes)
for m in messages:
    print(m)

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'No operation executed', 'Type ID': 3}
{'Type': 'Write', 'Type ID': 1, 'Device 

In [10]:
# re-write the whole configuration (program, parameters, control registers), equvalient to software reset.

dsp.control.reload_from_eeprom()

In [11]:
#  write a "Message" to hardware.
 
dsp.control.write_message(messages[0])
print(messages[0])

{'Type': 'Write', 'Type ID': 1, 'Device Address': 0, 'Sub Address': '0x081C', 'Data': b'\x00X'}


### XML related methods

In [12]:
import os
from sigma.sigma_studio.project.project_xml import *

project_xml_file_url = os.sep.join(['..', '..', 'SigmaStudio projects', 'projects', 'demo', 'demo.xml']) 
ic = get_IC(project_xml_file_url) 

In [13]:
# read SigmaStudio project XML file and re-write the whole configuration (program, parameters, control registers).

dsp.control.write_xml_ic(ic)

### Text file related methods

In [14]:
# load SigmaStudio files and re-write the whole configuration (program, parameters, control registers).

dsp.control.load_SigmStudio_files(file_NumBytes = 'NumBytes_IC_1.dat', file_TxBuffer = 'TxBuffer_IC_1.dat')